##Урок 8. Рекуррентные нейронные сети RNN LSTM GRU
Данные берем отызывы за лето

На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для нашей задачи

построить свёрточные архитектуры

построить различные архитектуры с RNN

построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd

import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from pymorphy2 import MorphAnalyzer

import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, Flatten, SimpleRNN, LSTM, GRU, Reshape
from keras.losses import categorical_crossentropy, SparseCategoricalCrossentropy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_excel('/content/drive/MyDrive/datasets/отзывы за лето.xls')
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [ ]:
max_words = 20000
max_len = 150
num_classes = 5

epochs = 20
batch_size = 512
print_batch_n = 100

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
stopwordslist = stopwords.words("russian")
ptrn = r'[^a-zA-Zа-яА-Я0-9]'

morpher = MorphAnalyzer()

def words_only(text):
    text=str(text)
    return text.lower()   

def remove_punkt(text):
    return re.sub(ptrn, ' ', text)

def to_token(text):
    return nltk.tokenize.word_tokenize(text)

def remove_stopwords(text):
    text_list = [w for w in text if w not in stopwordslist]
    return ' '.join(word for word in text_list)

def morphe_text(text):
    text = [morpher.parse(word)[0].normal_form for word in text.split() if word not in stopwordslist]
    return " ".join(text)

def normalize_text(text):
    text = words_only(text)
    text = remove_punkt(text)
    text = to_token(text)
    text = remove_stopwords(text)
    text = morphe_text(text)
    return text

In [ ]:
data['normalized_content'] = data['Content'].apply(normalize_text)
data.head()

,Rating,Content,Date,normalized_content
0,5,It just works!,2017-08-14,it just works
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложение минус хотеть слишко...
2,5,Отлично все,2017-08-14,отлично
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,стать зависать 1 работа антивирус далёкий нику...
4,5,"Очень удобно, работает быстро.",2017-08-14,очень удобно работать быстро


In [ ]:
train_corpus = " ".join(data['normalized_content'])
train_tokens = word_tokenize(train_corpus)
train_tokens_filtered = [word for word in train_tokens if word.isalnum()]

In [ ]:
dist = FreqDist(train_tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
voc = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [ ]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in voc:
            result.append(voc[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

data_train = np.asarray(
    [text_to_sequence(text, max_len) for text in data['normalized_content']], 
    dtype=np.int32)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_train, data.Rating, test_size=0.3, random_state=1)

le = LabelEncoder()

y_train = le.fit_transform(y_train) 
y_test = le.transform(y_test)

###CNN

In [ ]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(GlobalMaxPool1D())
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 42s 2s/step - loss: 1.4695 - accuracy: 0.0450 - val_loss: 1.3962 - val_accuracy: 0.7394
Epoch 2/5
23/23 [==============================] - 43s 2s/step - loss: 1.2012 - accuracy: 0.7341 - val_loss: 1.0539 - val_accuracy: 0.7549
Epoch 3/5
23/23 [==============================] - 42s 2s/step - loss: 0.8788 - accuracy: 0.7643 - val_loss: 0.8426 - val_accuracy: 0.7449
Epoch 4/5
23/23 [==============================] - 43s 2s/step - loss: 0.6986 - accuracy: 0.7754 - val_loss: 0.7930 - val_accuracy: 0.7608
Epoch 5/5
23/23 [==============================] - 41s 2s/step - loss: 0.6308 - accuracy: 0.7822 - val_loss: 0.7755 - val_accuracy: 0.7501


In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = pd.DataFrame(columns=['model', 'Test score', 'Test accuracy'])

metrics_df = metrics_df.append({
    'model': 'CNN',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 6s 495ms/step - loss: 0.8026 - accuracy: 0.7339
Test score: 0.8025774955749512
Test accuracy: 0.7339464426040649


<ipython-input-68-f8505f291b9a>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


###SimpleRNN

In [ ]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(SimpleRNN(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Reshape((1,128)))
model.add(SimpleRNN(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 28s 1s/step - loss: 1.2900 - accuracy: 0.6843 - val_loss: 0.8727 - val_accuracy: 0.7459
Epoch 2/5
23/23 [==============================] - 23s 1s/step - loss: 0.7827 - accuracy: 0.7496 - val_loss: 0.7182 - val_accuracy: 0.7698
Epoch 3/5
23/23 [==============================] - 25s 1s/step - loss: 0.6605 - accuracy: 0.7776 - val_loss: 0.6798 - val_accuracy: 0.7791
Epoch 4/5
23/23 [==============================] - 23s 1s/step - loss: 0.6073 - accuracy: 0.7924 - val_loss: 0.6819 - val_accuracy: 0.7764
Epoch 5/5
23/23 [==============================] - 25s 1s/step - loss: 0.5688 - accuracy: 0.8056 - val_loss: 0.6951 - val_accuracy: 0.7764


In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'SimpleRNN',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 3s 252ms/step - loss: 0.7218 - accuracy: 0.7567
Test score: 0.7217828035354614
Test accuracy: 0.7566956877708435


<ipython-input-70-726e1a524894>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


###LSTM

In [ ]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(LSTM(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Reshape((1,128)))
model.add(LSTM(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 89s 3s/step - loss: 1.5769 - accuracy: 0.6745 - val_loss: 1.5016 - val_accuracy: 0.7155
Epoch 2/5
23/23 [==============================] - 62s 3s/step - loss: 1.2500 - accuracy: 0.7030 - val_loss: 0.9622 - val_accuracy: 0.7155
Epoch 3/5
23/23 [==============================] - 62s 3s/step - loss: 0.8217 - accuracy: 0.7037 - val_loss: 0.7337 - val_accuracy: 0.7197
Epoch 4/5
23/23 [==============================] - 60s 3s/step - loss: 0.6908 - accuracy: 0.7192 - val_loss: 0.7040 - val_accuracy: 0.7197
Epoch 5/5
23/23 [==============================] - 63s 3s/step - loss: 0.6426 - accuracy: 0.7790 - val_loss: 0.6927 - val_accuracy: 0.7788


In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'LSTM',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 11s 859ms/step - loss: 0.7123 - accuracy: 0.7614
Test score: 0.7123333811759949
Test accuracy: 0.7613746523857117


<ipython-input-72-c7475f127a67>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


###GRU

In [ ]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(GRU(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Reshape((1,128)))
model.add(GRU(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 59s 2s/step - loss: 1.4819 - accuracy: 0.6756 - val_loss: 1.1961 - val_accuracy: 0.7155
Epoch 2/5
23/23 [==============================] - 49s 2s/step - loss: 0.9596 - accuracy: 0.7030 - val_loss: 0.8111 - val_accuracy: 0.7155
Epoch 3/5
23/23 [==============================] - 48s 2s/step - loss: 0.7394 - accuracy: 0.7091 - val_loss: 0.7093 - val_accuracy: 0.7715
Epoch 4/5
23/23 [==============================] - 48s 2s/step - loss: 0.6645 - accuracy: 0.7740 - val_loss: 0.6850 - val_accuracy: 0.7767
Epoch 5/5
23/23 [==============================] - 54s 2s/step - loss: 0.6195 - accuracy: 0.7848 - val_loss: 0.6769 - val_accuracy: 0.7788


In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'GRU',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 9s 685ms/step - loss: 0.7112 - accuracy: 0.7561
Test score: 0.7111561298370361
Test accuracy: 0.7560503482818604


<ipython-input-74-c7a60f594fef>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


# ###CNN -> LSTM

In [ ]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 76s 3s/step - loss: 1.3045 - accuracy: 0.6735 - val_loss: 1.0127 - val_accuracy: 0.7162
Epoch 2/5
23/23 [==============================] - 68s 3s/step - loss: 0.9118 - accuracy: 0.7226 - val_loss: 0.7740 - val_accuracy: 0.7535
Epoch 3/5
23/23 [==============================] - 69s 3s/step - loss: 0.7100 - accuracy: 0.7634 - val_loss: 0.6732 - val_accuracy: 0.7688
Epoch 4/5
23/23 [==============================] - 74s 3s/step - loss: 0.6248 - accuracy: 0.7822 - val_loss: 0.6571 - val_accuracy: 0.7760
Epoch 5/5
14/23 [=================>............] - ETA: 24s - loss: 0.5858 - accuracy: 0.7891

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'CNN -> LSTM',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

###CNN -> GRU

In [ ]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(GRU(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 63s 3s/step - loss: 1.1146 - accuracy: 0.6886 - val_loss: 0.8399 - val_accuracy: 0.7155
Epoch 2/5
23/23 [==============================] - 59s 3s/step - loss: 0.7758 - accuracy: 0.7183 - val_loss: 0.7039 - val_accuracy: 0.7573
Epoch 3/5
23/23 [==============================] - 63s 3s/step - loss: 0.6730 - accuracy: 0.7670 - val_loss: 0.6747 - val_accuracy: 0.7719
Epoch 4/5
23/23 [==============================] - 58s 3s/step - loss: 0.6094 - accuracy: 0.7841 - val_loss: 0.6751 - val_accuracy: 0.7698
Epoch 5/5
23/23 [==============================] - 59s 3s/step - loss: 0.5607 - accuracy: 0.8020 - val_loss: 0.6872 - val_accuracy: 0.7649


In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'CNN -> GRU',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 9s 736ms/step - loss: 0.7146 - accuracy: 0.7452
Test score: 0.7145793437957764
Test accuracy: 0.7452403903007507


<ipython-input-78-3cab87a72cc7>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


###CNN -> SimpleRNN

In [ ]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(SimpleRNN(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 47s 2s/step - loss: 1.1963 - accuracy: 0.6467 - val_loss: 0.9614 - val_accuracy: 0.7155
Epoch 2/5
23/23 [==============================] - 35s 2s/step - loss: 0.9664 - accuracy: 0.7030 - val_loss: 0.8531 - val_accuracy: 0.7155
Epoch 3/5
23/23 [==============================] - 39s 2s/step - loss: 0.8290 - accuracy: 0.7050 - val_loss: 0.7452 - val_accuracy: 0.7200
Epoch 4/5
23/23 [==============================] - 35s 2s/step - loss: 0.7242 - accuracy: 0.7402 - val_loss: 0.7060 - val_accuracy: 0.7518
Epoch 5/5
23/23 [==============================] - 43s 2s/step - loss: 0.6560 - accuracy: 0.7690 - val_loss: 0.6795 - val_accuracy: 0.7660


In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'CNN -> SimpleRNN',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 7s 553ms/step - loss: 0.7145 - accuracy: 0.7451
Test score: 0.7144688963890076
Test accuracy: 0.7450790405273438


<ipython-input-81-13369e4be48f>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


###SimpleRNN -> CNN

In [ ]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 56s 2s/step - loss: 1.3128 - accuracy: 0.5370 - val_loss: 1.0430 - val_accuracy: 0.7159
Epoch 2/5
23/23 [==============================] - 54s 2s/step - loss: 0.9359 - accuracy: 0.7249 - val_loss: 0.8281 - val_accuracy: 0.7421
Epoch 3/5
23/23 [==============================] - 57s 2s/step - loss: 0.7710 - accuracy: 0.7470 - val_loss: 0.7496 - val_accuracy: 0.7573
Epoch 4/5
23/23 [==============================] - 59s 3s/step - loss: 0.6886 - accuracy: 0.7649 - val_loss: 0.7124 - val_accuracy: 0.7667
Epoch 5/5
23/23 [==============================] - 58s 3s/step - loss: 0.6297 - accuracy: 0.7800 - val_loss: 0.7148 - val_accuracy: 0.7677


In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'SimpleRNN -> CNN',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 8s 626ms/step - loss: 0.7326 - accuracy: 0.7538
Test score: 0.7326456904411316
Test accuracy: 0.7537915706634521


<ipython-input-92-2d413a51352c>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


###LSTM -> CNN

In [ ]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(LSTM(128, return_sequences=True))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 95s 4s/step - loss: 1.2388 - accuracy: 0.6865 - val_loss: 0.9697 - val_accuracy: 0.7155
Epoch 2/5
23/23 [==============================] - 90s 4s/step - loss: 0.8513 - accuracy: 0.7030 - val_loss: 0.7707 - val_accuracy: 0.7155
Epoch 3/5
23/23 [==============================] - 105s 5s/step - loss: 0.7240 - accuracy: 0.7386 - val_loss: 0.7236 - val_accuracy: 0.7608
Epoch 4/5
23/23 [==============================] - 96s 4s/step - loss: 0.6698 - accuracy: 0.7685 - val_loss: 0.7025 - val_accuracy: 0.7660
Epoch 5/5
23/23 [==============================] - 98s 4s/step - loss: 0.6387 - accuracy: 0.7756 - val_loss: 0.7047 - val_accuracy: 0.7691


In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'LSTM -> CNN',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 14s 1s/step - loss: 0.7267 - accuracy: 0.7564
Test score: 0.7267023324966431
Test accuracy: 0.7563730478286743


<ipython-input-94-99076512101e>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


###GRU -> CNN

In [ ]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len,
                    trainable=True,
                    mask_zero=True))
model.add(GRU(128, return_sequences=True))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_split=0.2)

Epoch 1/5
23/23 [==============================] - 82s 3s/step - loss: 1.4315 - accuracy: 0.6843 - val_loss: 1.1226 - val_accuracy: 0.7176
Epoch 2/5
23/23 [==============================] - 74s 3s/step - loss: 0.9876 - accuracy: 0.7296 - val_loss: 0.8190 - val_accuracy: 0.7570
Epoch 3/5
23/23 [==============================] - 74s 3s/step - loss: 0.7211 - accuracy: 0.7656 - val_loss: 0.6865 - val_accuracy: 0.7719
Epoch 4/5
23/23 [==============================] - 74s 3s/step - loss: 0.6106 - accuracy: 0.7822 - val_loss: 0.6706 - val_accuracy: 0.7739
Epoch 5/5
23/23 [==============================] - 75s 3s/step - loss: 0.5597 - accuracy: 0.7916 - val_loss: 0.6863 - val_accuracy: 0.7743


In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

metrics_df = metrics_df.append({
    'model': 'GRU -> CNN',
    'Test score': score[0],
    'Test accuracy': score[1],
}, ignore_index=True)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 11s 836ms/step - loss: 0.7155 - accuracy: 0.7625
Test score: 0.715501070022583
Test accuracy: 0.7625040411949158


<ipython-input-96-1cb997e7e0ac>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


###Итог

In [ ]:
metrics_df.sort_values(by='Test score', ascending=False)

,model,Test score,Test accuracy
0,CNN,0.802577,0.733946
7,SimpleRNN -> CNN,0.732646,0.753792
8,LSTM -> CNN,0.726702,0.756373
1,SimpleRNN,0.721783,0.756696
9,GRU -> CNN,0.715501,0.762504
5,CNN -> GRU,0.714579,0.745240
6,CNN -> SimpleRNN,0.714469,0.745079
2,LSTM,0.712333,0.761375
3,GRU,0.711156,0.756050
4,CNN -> LSTM,0.701181,0.762343


In [ ]:
metrics_df.sort_values(by='Test accuracy', ascending=False)

,model,Test score,Test accuracy
9,GRU -> CNN,0.715501,0.762504
4,CNN -> LSTM,0.701181,0.762343
2,LSTM,0.712333,0.761375
1,SimpleRNN,0.721783,0.756696
8,LSTM -> CNN,0.726702,0.756373
3,GRU,0.711156,0.756050
7,SimpleRNN -> CNN,0.732646,0.753792
5,CNN -> GRU,0.714579,0.745240
6,CNN -> SimpleRNN,0.714469,0.745079
0,CNN,0.802577,0.733946
